In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc,  font_manager
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import statistics
matplotlib.rcParams['axes.unicode_minus'] = False
fontname = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=fontname).get_name()
rc('font', family=font_name)

In [2]:
get_data = pd.read_csv('../data/TEMP/경락가격/대파/대파 월별 데이터와 평균편차(전국포함).csv', index_col=None)

In [3]:
get_data.head()

,Date,서울 총거래물량,서울 총거래금액,서울 평균단가,서울 평균단가 편차,부산 총거래물량,부산 총거래금액,부산 평균단가,부산 평균단가 편차,대구 총거래물량,...,대전 총거래물량,대전 총거래금액,대전 평균단가,대전 평균단가 편차,광주 총거래물량,광주 총거래금액,광주 평균단가,광주 평균단가 편차,전국 평균단가,전국 평균단가 편차
0,2020-01-01,9462928.5,9136197340,965.472511,509.399083,1572788.4,1681745720,1069.276528,274.682556,766068.0,...,1655844.4,1592782952,961.915837,529.758006,931481.0,891210180,956.766891,412.771544,976.843293,468.247387
1,2020-02-01,8754610.0,7387886850,843.885319,630.986276,1647451.3,1372014542,832.810379,511.148705,689616.5,...,1528847.0,1236327688,808.666719,689.205381,870570.0,694122450,797.319515,572.218920,833.917135,611.173544
2,2020-03-01,9109415.0,6478744150,711.214074,763.657521,1696068.0,1191287642,702.382005,641.577080,487140.0,...,1641619.0,1162323521,708.034886,831.205448,884383.0,579553380,655.319449,714.218986,706.263606,738.827074
3,2020-04-01,8229718.0,6756209940,820.952788,653.918806,1635385.5,1165127230,712.448062,631.511023,596138.0,...,1355853.0,1180578684,870.727641,752.775000,831719.0,610273730,733.749896,635.788538,801.365040,643.725640
4,2020-05-01,7901230.0,9477792330,1199.533785,275.337809,1404740.2,1513407423,1077.357523,266.601561,413997.0,...,1477557.0,1817564652,1230.114745,345.889964,725731.0,827794130,1140.634932,228.903502,1176.068745,269.021934


In [4]:
get_data['전국 총거래물량'] = get_data['서울 총거래물량'] + get_data['부산 총거래물량'] + get_data['대구 총거래물량'] + get_data['대전 총거래물량'] + get_data['광주 총거래물량']
get_data['전국 총거래금액'] = get_data['서울 총거래금액'] + get_data['부산 총거래금액'] + get_data['대구 총거래금액'] + get_data['대전 총거래금액'] + get_data['광주 총거래금액']

In [5]:
seoul_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('서울')])
seoul_data = get_data[seoul_columns]

busan_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('부산')])
busan_data = get_data[busan_columns]

daegu_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대구')])
daegu_data = get_data[daegu_columns]

daejeon_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('대전')])
daejeon_data = get_data[daejeon_columns]

gwangju_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('광주')])
gwangju_data = get_data[gwangju_columns]

aways_columns = ['Date'] + list(get_data.columns[get_data.columns.str.contains('전국')])
aways_data = get_data[aways_columns]

In [6]:
aways_data = aways_data[['Date', '전국 총거래물량', '전국 총거래금액', '전국 평균단가', '전국 평균단가 편차']]

In [7]:
seoul_data = seoul_data.rename(columns={'서울 총거래물량':'총거래물량', '서울 총거래금액':'총거래금액', '서울 평균단가':'평균단가', '서울 평균단가 편차':'평균단가 편차'})
busan_data = busan_data.rename(columns={'부산 총거래물량':'총거래물량', '부산 총거래금액':'총거래금액', '부산 평균단가':'평균단가', '부산 평균단가 편차':'평균단가 편차'})
daegu_data = daegu_data.rename(columns={'대구 총거래물량':'총거래물량', '대구 총거래금액':'총거래금액', '대구 평균단가':'평균단가', '대구 평균단가 편차':'평균단가 편차'})
daejeon_data = daejeon_data.rename(columns={'대전 총거래물량':'총거래물량', '대전 총거래금액':'총거래금액', '대전 평균단가':'평균단가', '대전 평균단가 편차':'평균단가 편차'})
gwangju_data = gwangju_data.rename(columns={'광주 총거래물량':'총거래물량', '광주 총거래금액':'총거래금액', '광주 평균단가':'평균단가', '광주 평균단가 편차':'평균단가 편차'})
aways_data = aways_data.rename(columns={'전국 총거래물량':'총거래물량', '전국 총거래금액':'총거래금액', '전국 평균단가':'평균단가', '전국 평균단가 편차':'평균단가 편차'})

In [8]:
seoul_data['지역'] = '서울'
busan_data['지역'] = '부산'
daegu_data['지역'] = '대구'
daejeon_data['지역'] = '대전'
gwangju_data['지역'] = '광주'
aways_data['지역'] = '전국'

In [9]:
all_data = pd.concat([seoul_data, busan_data, daegu_data, daejeon_data, gwangju_data, aways_data])

In [10]:
all_data.to_csv('../data/TEMP/경락가격/대파/대파 컬럼 통합한 데이터와 표준편차.csv', index=False, encoding='UTF-8')